# ***rfpred*** 🧪

Welcome to our documentation notebook for the package ***rfpred***, a package to predict the Rf value of silica thin-layer chromatographies based on the experimental conditions used in the lab!

In this notebook we will guide you through our progress and discuss, which challenges we faced along the way. Below you find the table of contents to get a glimpse on the global structure of our project.

***Table of Contents***

1. Introduction
2. Extraction of data
3. Pre-Processing of the Data using Regex and a LLM
4. Cleaning of the data
5. Training the model (notebooks2)
6. Streamlit GUI and input processing (notebooks2)


# 1. Introduction 🎯 <a name="introduction"></a>

TLC is a common practice in synthesis labs. However, spotting THE spot (the product) is often harder as we might think due to the oftentimes huge amount of by-products in new reaction discovery. With this project, we aim to facilitate chemists lives. We trained a model on data that we extracted from the dataset of chemical reactions from US patent applications from 2001 to 2016, collected by Daniel Lowe. This dataset can be found here: 🔍 https://figshare.com/articles/dataset/Chemical_reactions_from_US_patents_1976-Sep2016_/5104873. 

_Now let's dive in a bit deeper._

# 2. Extraction of data 🛠️
As there is no open source dataset on Rf values, we had to extract the relevant information from the XML files of the US patents dataset.
We were interested in the title, the general experimental procedure, the reaction SMILES, but most importantly the product SMILES. That is when we faced the first challenge: we did not know, how to treat and iterate through XML files. This turned out to be pretty feasible after spending a bit of time, familiarizing with the structure. However, we noted that not all information we wanted to extract where listed in the same manner (the product SMILES for example), so it took a while until we finalized the function that enabled us to extract all the data we wanted to have. 


In [2]:
# First we import the necessary libraries

import xml.etree.cElementTree as et     # for parsing the XML file
import pandas as pd
import numpy as np
import os
import re
from groq import Groq # for the LLM Groq queries (needs to be installed via pip)
import json
from tqdm import tqdm
from config import LLM_API_KEY
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import rdChemReactions, rdmolops
from rdkit.Chem.Draw import IPythonConsole

Now we use a function to iterate over all the XML files in the applications data from 2001. We collect the title, the paragraphText, mostly including the experimental procedures, the reaction SMILES and the product SMILES from it.

In [3]:
def extract_data(root_file):
    """function that extracts data from an XML file and returns a list of dictionaries containing the extracted information.
        Information to be extracted includes the title of the reaction, the experimental procedure, the reaction SMILES, and the product SMILES.

    Args:
        root_file: root of the parsed XML file

    Returns:
        reaction_list: list of dictionaries containing the extracted information
    """
    
    # Define the namespace that is used to prevent mismatching of tags in the XML file
    ns = {'cml': 'http://www.xml-cml.org/schema', 'dl': 'http://bitbucket.org/dan2097'}

    # Create lists to store extracted information
    reaction_list = []

    # Find all <reaction> elements
    reaction_elements = root_file.findall('.//cml:reaction', ns)

    # Iterate over each <reaction> element
    for reaction_element in reaction_elements:
        # Create a dictionary to store information about the reaction
        reaction_dict = {}

        # Extract title
        title = reaction_element.find('.//dl:headingText', ns)
        if title is not None:
            reaction_dict['title'] = title.text

        # Extract paragraph text
        paragraph_text = reaction_element.find('.//dl:paragraphText', ns)
        if paragraph_text is not None:
            reaction_dict['paragraphText'] = paragraph_text.text

        # Extract reaction SMILES
        reaction_smiles = reaction_element.find('.//dl:reactionSmiles', ns)
        if reaction_smiles is not None:
            reaction_dict['reactionSmiles'] = reaction_smiles.text

        # Extract productSMILES
        product_elements = reaction_element.findall('.//cml:product', ns)
        product_smiles = []
        for product_element in product_elements:
            identifier_element = product_element.find('.//cml:identifier[@dictRef="cml:smiles"]', ns)
            if identifier_element is not None:
                smiles_value = identifier_element.get('value')
                if smiles_value is not None:
                    product_smiles.append(smiles_value)
        if product_smiles:
            reaction_dict['productSmiles'] = product_smiles

        # Append reaction dictionary to reaction list
        reaction_list.append(reaction_dict)

    # Check if any values were extracted
    #print("Reaction List:", reaction_list)
    return reaction_list


In a second step, we iterate through all files in every folder of the application data of the dataset and extract the data with the function `extract_data`. This takes quite a bit, but you can see the progress whenever a new folder is treated.

In [ ]:
Applications_list = []

# define path to access first XML file in the folder 2001 of applications
# Insert the path to the folder containing the XML files
for folder in os.listdir(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications'):
    folder = os.path.join(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\applications', folder)
    print(folder) 
    for file in os.listdir(folder):
        if file.endswith('.xml'):
            file = os.path.join(folder, file)
            tree = et.parse(file)
         # define root of the XML file to iterate through the file
            root = tree.getroot()
            Applications_list.append(extract_data(root))    
                  

After the first step, it was clear that the extracted data needed a lot (a LOT) of pre-processing before we could finally feed it into our ML model. We decided, it would be smartest to convert everything into a pandas dataframe from the list of dictionaries we got from the `extract_data` function.

In [ ]:
# Create a DataFrame from the extracted data
df_extracts = pd.DataFrame(Applications_list[0])

# Iterate through the list of dictionaries and create a DataFrame
for i in range(1, len(Applications_list)):
    df = pd.DataFrame(Applications_list[i])
    
    # Concatenate the DataFrames
    df_extracts = pd.concat([df_extracts, df], ignore_index=True)

print(df_extracts.shape)
df_extracts.head()


(1939253, 4)


,paragraphText,reactionSmiles,productSmiles,title
0,"PL 137,526 describes the hydrogenation of p-te...",[C:1]([C:5]1[CH:10]=[CH:9][C:8]([OH:11])=[CH:7...,[C(C)(C)(C)C1CCC(CC1)O],NaN
1,"Slurry aluminum chloride (140.9 g, 1.075 mol) ...",[Cl-].[Al+3].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C(C)C],Step h: 4-Chloro-1-(4-isopropyl-phenyl)-butan-...
2,"Suspend anhydrous AlCl3 (156 g, 1.15 mol) in t...",[Al+3].[Cl-].[Cl-].[Cl-].[Cl:5][CH2:6][CH2:7][...,[ClCCCC(=O)C1=CC=C(C=C1)C],Step d: 4-Chloro-1-(4-methyl-phenyl)-butan-1-one
3,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...
4,Dissolve 4-chloro-1-(4-isopropyl-phenyl)-butan...,[Cl:1][CH2:2][CH2:3][CH2:4][C:5]([C:7]1[CH:12]...,[BrC(C)(C)C1=CC=C(C=C1)C(CCCCl)=O],1-[4-(1-Bromo-1-methyl-ethyl)-phenyl]-4-chloro...


That Dataframe is huge! Finally, save all the extracted data into a csv file on the local device.

In [ ]:
df_extracts.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv', index=False)

# 3. Pre-Processing of the Data using Regex and a LLM 💬 <a name = "PreProcessing"></a>
Of course, not all entries in the data frame can be used for our model. Many of the experimental procedures do not include any information about the Rf value. Thus, we need to find the entries with Rf values. There is only one problem: all experimental procedures are written differently. Thus, filtering them with Regex is quite a pain and did not give the outcome, we envisioned (although we spent enugh time with regex to call us friends by now). To circumvent this, we created a function, that pre-filters the experimental procedure to drop the rows that do not include any information about Rf values and keeps the other entries. Those remaining entries - as they are very broadely filtering - are treated with a LLM (if you want to run it you have to sign up for Groq and insert your API key and define it in the function) and the response is parsed with regex again to extract the information we need. 

The `extract_rows_with_rf` function is our not very selective pre-filter, to at least slightly reduce the computation cost of the LLM afterwards.

In [3]:
def extract_rows_with_rf(Dataframe: pd.DataFrame):
    """Function that extracts rows with Rf values from a dataframe and returns a new dataframe containing only these rows. 

    Args:
        Dataframe (_type_): Dataframe containing the extracted data from the US patents
    """
    # copy the dataframe to leave old dataframe unchanged
    df = Dataframe.copy()
    
    # Define the regex patterns
    Rf_check = r'( ?R[fF][ :=(]?)'
    
    # List to store indices of rows without Rf values
    rows_to_drop = []
    
    # Search for rows with Rf values in the paragraphText column
    for index, row in df.iterrows():
        checkRf = re.findall(Rf_check, row['paragraphText'])

        if not checkRf:
            rows_to_drop.append(index)
               
    # Drop rows without Rf values
    df = df.drop(rows_to_drop)
            
    return df
    

Now we come to the heart of the pre-processing: the `get_values` and `parse_response` functions iterate through the whole dataframe and apply the LLM to a very specific user prompt we wrote. Before coming up with this solution, to again use regex for the extraction of the solvents and ratios, we tried to directly convert the response of the LLM into a dictionary which could have been easily added to the new dataframe. However, we had to accept, that LLMs love talking bla bla and this blew up our rigid structure which was needed to avoid errors. So, again, regex, our old friend. 

In [5]:
def parse_response(response: str):
    """Function that extracts the Rf value, solvent A, solvent B, % solvent A, and % solvent B from a LLM response.

    Args:
        response (str): response from the LLM model 

    Returns:
        rf_value (str)
        solvent_a (str) 
        solvent_b (str) 
        percent_a (str)
        percent_b (str)
        additive_c (str)
        percent_c (str)
    """
    try: 
        
    # search for Rf value in the response using regex
        rf_value_match = re.search(r"Rf\s*=\s*(\d+\.\d+)", response)
        solvent_a_match = re.search(r"solvent\s+A\s*=\s*([^,]+)", response)
        solvent_b_match = re.search(r"solvent\s+B\s*=\s*([^,]+)", response)
        additive_c_match = re.search(r"additive\s+C\s*=\s*([^,]+)", response)
        percent_a_match = re.search(r"percent\s+A\s*=\s*(\d+)", response)
        percent_b_match = re.search(r"percent\s+B\s*=\s*(\d+)", response)
        percent_c_match = re.search(r"percent\s+C\s*=\s*(\d+\.\d+|\d+)", response)
        
    except KeyError:
        raise KeyError(f"Error extracting values from the response: {response}")
    
    # Extract values from regex matches
    rf_value = rf_value_match.group(1) if rf_value_match else None
    solvent_a = solvent_a_match.group(1) if solvent_a_match else None
    solvent_b = solvent_b_match.group(1) if solvent_b_match else None
    percent_a = percent_a_match.group(1) if percent_a_match else None
    percent_b = percent_b_match.group(1) if percent_b_match else None
    additive_c = additive_c_match.group(1) if additive_c_match else None
    percent_c = percent_c_match.group(1) if percent_c_match else None

    # Return the extracted values
    return rf_value, solvent_a, solvent_b, percent_a, percent_b, additive_c, percent_c

In [34]:
def get_values(Dataframe: pd.DataFrame):
    """Function that extracts the Rf value from a row in a Dataframe using LLM and returns it.

    Args:
        DataFrame (_type_): DataFrame containing the extracted data from the US patents
    """
    
    client = Groq(
        api_key= LLM_API_KEY ) # insert your API Key for Groq in the config.py file to use the LLM model
    
    for index, row in tqdm(Dataframe.iterrows(), total=Dataframe.shape[0]):
        user_prompt = f"""Give me the Rf value, the solvent mixture and the ratio of the solvents in percent for each solvent of this 
                        procedure: {row['paragraphText']} in the following format only: Rf= , solvent A= , solvent B= , percent A= , 
                        percent B= . If there is a third solvent, please provide the information in the same format, call it additive C =
                        and percent C = . If no information is given for one of the categories, put None for this category. Only give the 
                        information requested as output, no additional notes or information!!!""" # f"Please only provide the Rf value, solvents A and B, additives C (if applicable), and their ratios in percent as a Python dictionary only (without 'Here is the python dictionary') with keys 'Rf', 'solvent A', 'solvent B', 'additive C', 'percent A', 'percent B', and 'percent C' of this procedure: {row['paragraphText']}. If there is no information for one category, put 'Nan'. Do not provide any additional notes or information except for the dictionary!!! Always use this format!!" # "Give me the Rf value, the solvent A, B and additive C if applied and their ratios in percent as a python dictionary calling the keys Rf (should be a number), solvent A (should be a solvent only), solvent B (solvent only), percent A (number), percent B (number), additive C (solvent), percent C (number) of this procedure:" + row['paragraphText'] + "If there is no information for one category of solvent, put Nan. Do not put any other notes or information except for the dictionary!!! Always use this format!!"  # "Give me the Rf value, the solvent mixture and their ratio of the following procedure" + row['paragraphText'] +  "in the following format of a dictionary only: Rf= , solvent A= , solvent B= , percent A= , percent B= . If there is a third solvent, please provide the information in the same format, call it additive C and percent C = . Only give the values requested as output, no additional notes or information!!!"        
        
        try:             
            # Call the LLM model to extract the Rf value
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
                model="llama3-70b-8192", # other models: LlaMA3 70 b (llama3-70b-8192), mixtral (mixtral-8x7b-32768) can be found here: https://console.groq.com/docs/models
            )
            
            # Extract the Rf value from the response
            response = chat_completion.choices[0].message.content
            #print(response)
            
            # track the progress of the function 
            if index % 100 == 0:
                print("Progress:", (index+1)/len(Dataframe) * 100, "%" )
            
            # Parse the response to extract the Rf value, solvent A, solvent B, % solvent A, and % solvent B using the parse_response function
            rf_value, solvent_a, solvent_b, percent_a, percent_b, additive_c, percent_c = parse_response(response)
            
            # Add extracted values to new columns in the dataframe row
            Dataframe.at[index, 'Rf'] = rf_value
            Dataframe.at[index, 'Solvent_A'] = solvent_a
            Dataframe.at[index, 'Solvent_B'] = solvent_b
            Dataframe.at[index, 'Percent_A'] = percent_a
            Dataframe.at[index, 'Percent_B'] = percent_b
            Dataframe.at[index, 'Additive_C'] = additive_c
            Dataframe.at[index, 'Percent_C'] = percent_c
            
            
            
        except Exception as e:
            # Print the error message and the index of the row where the error occurred
            print(e)
            print("Error at index:", index)
            continue
        
    return Dataframe
        

        
    

### 3.1 Pre-Processing with the extracted data
Now let's see, what we get out from the first pre-processing step. We wouldn't recommend you to run this following lines, as our laptops spent days and nights extracting row per row. 

First we load the previously extracted data from the US patents into a new dataframe and check its size (just out of curiosity).

In [4]:
# That is the csv file that contains all the raw data from the US patents
df_new = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\extracted_data_raw_applications.csv') # change directory here if necessary
df_new.shape

(1939253, 4)

Now we apply the first function `extract_rows_with_rf` to prefilter the dataframe. Already by this first filter, the size of the datafram shrinks quite a bit!

In [16]:
df_rf = extract_rows_with_rf(df_new)
print(df_rf.shape)
df_rf.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\extracted_data_second_processing_rf_values.csv', index=False)
df_rf.head()

,paragraphText,reactionSmiles,productSmiles,title
524,A mixture of 7-trifluoromethylsulfonyloxy-1-(4...,FC(F)(F)S(O[C:7]1[CH:16]=[C:15]2[C:10]([CH:11]...,['CN1CCN(CC1)C1=CC=CC2=CC=C(C=C12)C=1C=NC=NC1'],1-(4-Methylpiperazin-1-yl)-7-(pyrimid-5-yl)nap...
525,"A mixture of the above acid (0.25 g, 0.82 mmol...",[CH3:1][N:2]1[CH2:7][CH2:6]O[CH2:4][CH2:3]1.O....,['ClC1=CC=C(CNC(=O)C2=CC3=C(C=CC=C3C=C2)N2CCN(...,8-(4-Methylpiperazin-1-yl)naphthalene-2-carbox...
538,"A solution of 1,4-benzodioxan-6-amine (2.00 g,...",[O:1]1[C:6]2[CH:7]=[CH:8][C:9]([NH2:11])=[CH:1...,['O1CCOC2=C1C=CC(=C2)N2CCNCC2'],NaN
539,A solution of 6-aminochroman from part 2 of th...,[NH2:1][C:2]1[CH:3]=[C:4]2[C:9](=[CH:10][CH:11...,['O1CCCC2=CC(=CC=C12)N1CCNCC1'],1-(Chroman-6-yl)piperazine
540,"A solution of 1-(1,4-benzodioxan-6-yl)piperazi...",O1C2C=CC(N3CCNCC3)=CC=2OCC1.C(O)(=O)C(O)=O.[O:...,['O1CCOC2=C1C=CC(=C2)N2CCN(CC2)CC2=CC=C(C=C2)F'],NaN


As the entire dataframe so far is still too big and it takes too long to compute everything, we split down the dataframe into four sub-files and computed them simultaneously. Still a long time and some days in class without the laptops, as it was still computing. Google Colab was also not an option, as it automatically stopped after 6h and we were to greedy to pay for the upgrade...

In [6]:
df_4 = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\AfterRFfilter(4).csv')

After 25h of computing...

In [15]:
df_4_processed = get_values(df_4)
df_4_processed.head()

  0%|          | 1/9147 [00:01<4:49:08,  1.90s/it]

Progress: 0.010932546190007653 %


  1%|          | 66/9147 [11:21<20:37:32,  8.18s/it]

upstream connect error or disconnect/reset before headers. reset reason: remote connection failure, transport failure reason: delayed connect error: 111
Error at index: 65


  1%|          | 101/9147 [17:00<20:44:20,  8.25s/it]

...

Progress: 98.40384825625888 %


 99%|█████████▉| 9101/9147 [23:24:51<07:10,  9.37s/it]

Progress: 99.49710287525964 %


100%|██████████| 9147/9147 [23:31:52<00:00,  9.26s/it]


,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C
0,To a flame-dried round-bottomed flask was adde...,[CH2:1]([OH:11])[CH2:2][CH2:3][CH2:4][CH2:5][C...,['O1C(CCCC1)OCCCCCCCCC#C'],NaN,0.30,EtOAc,Hex,5,95,None,None
1,A flame-dried round-bottomed flask was charged...,[O:1]1[CH2:6][CH2:5][CH2:4][CH2:3][CH:2]1[O:7]...,['COC(C#CCCCCCCCCOC1OCCCC1)=O'],NaN,0.33,EtOAc,Hex,10,90,None,None
2,To a flame-dried round-bottomed flask equipped...,N(C(C)C)C(C)C.[Li]CCCC.[SnH:13]([CH2:22][CH2:2...,['O1C(CCCC1)OCCCCCCCC\\C(=C/C(=O)OC)\\[Sn](CCC...,NaN,0.30,EtOAc,Hex,5,95,None,None
3,To a flamed-dried round-bottomed flask was add...,[O:1]1[CH2:6][CH2:5][CH2:4][CH2:3][CH:2]1[O:7]...,['O1C(CCCC1)OCCCCCCCC\\C(=C/C(=O)OC)\\C(=C\\C(...,NaN,0.19,EtOAc,Hex,10,90,None,None
4,A flame-dried round-bottomed flask was charged...,[O:1]1[CH2:6][CH2:5][CH2:4][CH2:3][CH:2]1[O:7]...,['O1C(CCCC1)OCCCCCCCCC(CC(=O)OC)=C(CC(=O)OC)CC...,NaN,0.41,EtOAc,Hex,20,80,None,None


Finally got the resulting dataframe and quickly saved it!

In [16]:
df_4_processed.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\AfterRFfilter(4)_processed.csv', index=False)

In [17]:
df_4_processed.shape

(9147, 11)

# 4. Cleaning of the Data 🧹
Now we created new rows for the extracted values. In the following, we need to process the data further, to first drop the rows that don't include an Rf and were not filtered out in the first function, to convert the types from `str` to `int` for the Rf values and check, if the percent add up to 100 %, to see which solvents are used to create their corresponding SMILES and later molecular descriptors using `RDkit`. Furthermore, in some procedures, a third additive is added to the solvent mixture, which makes the data even more complex and on top of that we found out, that the calculation of the solvent ratio in percent often is not matching for those cases. To simplify the dataset, we thus decided to filter out those entries with a third additive. Lastly, the LLM partly took retention times from column chromatography as Rf values, so to filter out those, we exclude Rf values >1. However, when generally testing the performance of the LLM in combination with regex extraction, in the examples we checked (taken out the exceptions that were previously mentioned and recorded) it worked perfectly well.

### 4.1 Functions for the clean up
This first function `clean_up` does exactly what described in the previous paragraph. Over a bit of time and after some testing, we finalized this function to its current form to exclude all eventualities. Ater this we still had the problem of converting the solvents to some form, that can be fed into a ML model for training. As we knew from some cases of the previous tests we did with smaller dataframes, the LLM sometimes creates weird responses which don't meet the column standards we wanted. That's when we decided to create the `get_solvents` function, to not only get an overview of which solvents were frequently used but also an overview of potential errors that could occur. Based on this list that we got, we modified the `convert_solvents` function until it was bulletproof from our point of view. 

In [11]:
def clean_up(Dataframe: pd.DataFrame, tolerance: float = 1):
    """1. Delets all row whitout Rf value or Rf value over 1.0 and without solvent information.
       2. Converts percentage of solvents in a Dataframe from str to float and check if they add up to 100, else drop them.
       3. Strips productSMILES str to a usable SMILES str.

    Args: 
        Dataframe (_type_): Dataframe containing the processed data from the get_value function.
        Dataframe needs to have following columns: 'productSmiles,' 'Rf', 'Percent_A', 'Percent_B'
        tolerance (_type_): float, default = 1, tolerance for the sum of the percentages of the solvents (default is 100% +- 1%)

    """
    size_pre_cleaning = Dataframe.shape[0]  # get the size of the dataframe
    # Drop rows without Rf values
    Dataframe.dropna(subset=['Rf'], inplace = True) 
    
    # Convert Rf value to float
    Dataframe['Rf'] = Dataframe['Rf'].astype(float)
    
    #finds indicies with Rf values over 1.0
    indices_false_Rf = Dataframe[Dataframe["Rf"] > 1].index 
    Dataframe.drop(indices_false_Rf,inplace = True) #drops rows with false Rf values
    
    #check if at least one solvent (either solvent A or solvent B) is given (so check that solvent A and solvent B are not None)
    Dataframe = Dataframe[Dataframe['Solvent_A'].notnull() & Dataframe['Solvent_B'].notnull()].copy()
    
    #convert 'None' entry to 0
    Dataframe.loc[:, 'Percent_A'] = Dataframe['Percent_A'].apply(lambda x: 0 if x is None else x)
    
    #convert 'None' entry to 0
    Dataframe.loc[:, 'Percent_B'] = Dataframe['Percent_B'].apply(lambda x: 0 if x is None else x)
    
    # convert Percentage to float
    Dataframe.loc[:, 'Percent_A'] = Dataframe['Percent_A'].apply(lambda x: float(x)) 
    
    #convert Percantage to float
    Dataframe.loc[:, 'Percent_B'] = Dataframe['Percent_B'].apply(lambda x: float(x)) 
    
    # Drop rows where 'additive_C' is not None
    Dataframe = Dataframe[Dataframe['Additive_C'].isnull()].copy()
    
    # remove [' and '] from the productSimles
    Dataframe.loc[:, 'productSmiles'] = Dataframe['productSmiles'].apply(lambda x: x[1:-1]) 
    
    # check if the sum of the percentages is 100, this at the same time kicks out entries with additives C (+ consider limitations of floating-point arithmetic in the cases of 33.3333 and 66.667)
    Dataframe.loc[:, 'sum'] = Dataframe['Percent_A'] + Dataframe['Percent_B']
    Dataframe = Dataframe[(Dataframe['sum'] >= 100 - tolerance) & (Dataframe['sum'] <= 100 + tolerance)].copy()
    
    Dataframe.reset_index(drop=True, inplace=True) # sets the index new from 1 to end

    size_post_cleaning = Dataframe.shape[0]  # get the size of the dataframe after cleaning
    print(f"Size of the dataframe before cleaning: {size_pre_cleaning}")
    print(f"Size of the dataframe after cleaning: {size_post_cleaning}")
    print(f"Number of rows dropped: {size_pre_cleaning - size_post_cleaning}")
    print(f"Percentage of rows dropped: {(size_pre_cleaning - size_post_cleaning) / size_pre_cleaning * 100}%")
    return Dataframe


In [12]:
def get_solvents(Dataframe: pd.DataFrame):
    """Function that iterates over all solvent entries in the dataframe and checks, if the solvent mentioned is already in the list. If not, it adds the solvent to the list.

    Args:
        Dataframe (_type_): dataframe with the extracted data from the US patents, preprocessed with the get_values function.
    """
    solvents = []
    
    for index, row in Dataframe.iterrows():
        solvent_a = row['Solvent_A']
        solvent_b = row['Solvent_B']
        
        if solvent_a not in solvents and solvent_a is not None:
            solvents.append(solvent_a)
        if solvent_b not in solvents and solvent_b is not None:
            solvents.append(solvent_b)
        else:
            continue
        
    return solvents

In [13]:
def convert_solvents(Dataframe: pd.DataFrame):
    """Converts selected solvent names (based on frequency) in the dataframe to a standardized form.

    Args:
        Dataframe (pd.DataFrame): Dataframe containing the extracted data from the US patents, 
                                preprocessed with the get_values and clean_up function.
                                
    Returns:
        Dataframe (pd.DataFrame): Dataframe with standardized solvent names and SMILES.
        df_sorted_out (pd.DataFrame): Dataframe containing rows with solvents that could not be converted.
    """
    Dataframe = Dataframe.copy()
    df_sorted_out = pd.DataFrame()
    indices_to_drop = []
    size_pre_conversion = Dataframe.shape[0]  # get the size of the dataframe
    
    for index, row in Dataframe.iterrows():
        for solvent_column in ['Solvent_A', 'Solvent_B']: # iterate through the solvent columns
            solvent_name = row[solvent_column].lower()
            
            if '/' in solvent_name or '%' in solvent_name or '(' in solvent_name or ')' in solvent_name: # if there is a mixture of solvents, take out the entire row
                df_sorted_out = pd.concat([df_sorted_out, pd.DataFrame(row).transpose()], ignore_index=True)
                indices_to_drop.append(index) # add the index to the list of indices to drop not to change size of Dataframe with every iteration
                
            elif 'dcm' in solvent_name or 'dichlorometh' in solvent_name or 'methylene chloride' in solvent_name or 'ch2cl2' in solvent_name and 'chloroform' not in solvent_name and 'trichloromethane' not in solvent_name: # exclude chloroform and trichloromethane for substring search
                Dataframe.at[index, solvent_column] = 'DCM'     
                Dataframe.at[index, solvent_column + '_Smiles'] = 'ClCCl' # add SMILES of the solvent A 
                
            elif 'meoh' in solvent_name or 'methanol' in solvent_name or 'ch3oh' in solvent_name:
                Dataframe.at[index, solvent_column] = 'MeOH'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CO'
                
            elif 'acetonitrile' in solvent_name or 'acetonitril' in solvent_name or 'mecn' in solvent_name or 'ch3cn' in solvent_name or 'acn' in solvent_name:
                Dataframe.at[index, solvent_column] = 'MeCN'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CC#N'
                
            elif 'toluene' in solvent_name:
                Dataframe.at[index, solvent_column] = 'Toluene'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'Cc1ccccc1'
                
            elif 'hexane' in solvent_name or 'hex' in solvent_name or 'n-hex' in solvent_name or 'hexanes' in solvent_name and 'cycl' not in solvent_name: # exclude cyclohexane
                Dataframe.at[index, solvent_column] = 'Hexane'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCCCCC'
                
            elif 'chloroform' in solvent_name or 'trichloromethane' in solvent_name or 'chcl3' in solvent_name:
                Dataframe.at[index, solvent_column] = 'Chloroform'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'ClC(Cl)Cl'
                
            elif 'acetone' in solvent_name or 'me2co' in solvent_name or 'ch3coch3' in solvent_name or '2-propanone' in solvent_name or 'dimethyl ketone' in solvent_name:
                Dataframe.at[index, solvent_column] = 'Acetone'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CC(=O)C'
                
            elif 'etoh' in solvent_name or 'ethanol' in solvent_name or 'ch3ch2oh' in solvent_name or 'alcohol' in solvent_name:
                Dataframe.at[index, solvent_column] = 'EtOH'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCO'
                
            elif 'ether' in solvent_name or 'diethylether' in solvent_name or 'et2o' in solvent_name or 'diethyl ether' in solvent_name:
                Dataframe.at[index, solvent_column] = 'diethyl ether'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCOCC'
                
            elif 'hep' in solvent_name or 'heptane' in solvent_name or 'n-heptane' in solvent_name or 'heptanes' in solvent_name:
                Dataframe.at[index, solvent_column] = 'heptane'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCCCCCC'
                
            elif 'petroleum ether' in solvent_name or 'light petroleum' in solvent_name or 'pe' in solvent_name or 'pet ether' in solvent_name or 'pet. ether' in solvent_name or 'pet.ether' in solvent_name or 'petroleum ether (bp 40-60)' in solvent_name or 'petroleum ether (bp 60-80)' in solvent_name or 'petroleum ether (bp 80-100)' in solvent_name or 'petroleum ether (bp 100-140)' in solvent_name or 'petroleum ether (bp 140-180)' in solvent_name or 'petroleum ether (bp 180-210)' in solvent_name or 'petroleum ether (bp 210-240)' in solvent_name or 'petroleum ether (bp 240-270)' in solvent_name or 'petroleum ether (bp 270-300)' in solvent_name or 'petroleum ether (bp 300-400)' in solvent_name or 'petroleum ether (bp 400-600)' in solvent_name or 'petroleum ether (bp 600-800)' in solvent_name or 'petroleum ether (bp 800-1000)' in solvent_name or 'petroleum ether (bp 1000-1200)' in solvent_name or 'petroleum ether (bp 1200-1400)' in solvent_name or 'petroleum ether (bp 1400-1600)' in solvent_name or 'petroleum ether (bp 1600-1800)' in solvent_name or 'petroleum ether (bp 1800-2000)' in solvent_name or 'petroleum ether (bp 2000-2200)' in solvent_name or 'petroleum ether (bp 2200-2400)' in solvent_name or 'petroleum ether (bp 2400-2600)' in solvent_name or 'petroleum ether (bp 2600-2800)' in solvent_name or 'petroleum ether (bp 2800-3000)' in solvent_name or 'petroleum ether (bp 3000-3200)' in solvent_name or 'petroleum ether (bp 3200-3400)' in solvent_name or 'petroleum ether (bp 3400-3600)' in solvent_name or 'petroleum ether (bp 3600-3800)' in solvent_name or 'petroleum ether (bp 3800-4000)' in solvent_name or 'petroleum ether (bp 4000-4200)' in solvent_name:
                Dataframe.at[index, solvent_column] = 'petroleum ether (2-methylpentane)'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCCC(C)C'
                
            elif 'triethylamine' in solvent_name or 'et3n' in solvent_name or 'tea' in solvent_name:
                Dataframe.at[index, solvent_column] = 'triethylamine'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'CCN(CC)CC'
                
            elif 'etoac' in solvent_name or 'ethyl acetate' in solvent_name or 'ethylacetate' in solvent_name or 'ethylacetat' in solvent_name or 'ea' in solvent_name or 'ethanol acetate' in solvent_name:
                Dataframe.at[index, solvent_column] = 'EtOAc'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'O=C(OCC)C' 
                
            elif 'thf' in solvent_name or 'tetrahydrofuran' in solvent_name:
                Dataframe.at[index, solvent_column] = 'THF'
                Dataframe.at[index, solvent_column + '_Smiles'] = 'C1CCOC1'
                
            elif 'none' in solvent_name:
                Dataframe.at[index, solvent_column + '_Smiles'] = None
                
            else: # if the solvent is not in the list, add it to the other dataframe but still sort it out. We don't want things to get too complex.
                df_sorted_out = pd.concat([df_sorted_out, pd.DataFrame(row).transpose()], ignore_index=True)
                indices_to_drop.append(index) # add the index to the list of indices to drop not to change size of Dataframe with every iteration

    Dataframe = Dataframe.drop(indices_to_drop)
    Dataframe.reset_index(drop=True, inplace=True)
    df_sorted_out.reset_index(drop=True, inplace=True)
    
    size_post_conversion = Dataframe.shape[0]  # get the size of the dataframe after conversion to see how many rows were dropped
    print(f"Size of the dataframe before conversion: {size_pre_conversion}")
    print(f"Size of the dataframe after conversion: {size_post_conversion}")
    print(f"Number of rows dropped: {size_pre_conversion - size_post_conversion}")
    print(f"Percentage of rows dropped: {(size_pre_conversion - size_post_conversion) / size_pre_conversion * 100}%")
        
    return Dataframe, df_sorted_out

### 4.2 Processing of the large dataset
After some trying out and testing on smaller datasets to perfectionalize the `convert_solvents` function, we processed the big dataset that we recombined after we had split it for the LLM extraction.


In [29]:
df = pd.read_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\After_LLM.csv')

# Clean up the dataframe
df = clean_up(df)
print(f"shape:", df.shape)

# convert the solvents into standardized form and add SMILES
df, df_sorted_out = convert_solvents(df)
print(f"shape:", df.shape)
df.head(10)

Size of the dataframe before cleaning: 36123
Size of the dataframe after cleaning: 16617
Number of rows dropped: 19506
Percentage of rows dropped: 53.998837305871604%
shape: (16617, 13)
Size of the dataframe before conversion: 16617
Size of the dataframe after conversion: 15403
Number of rows dropped: 1214
Percentage of rows dropped: 7.3057711981705475%
shape: (15403, 15)


,Unnamed: 0,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C,sum,Solvent_A_Smiles,Solvent_B_Smiles
0,2,A solution of 4-{[(phenylmethoxy)carbonylamino...,[C:1]1([CH2:7][O:8][C:9]([NH:11][CH2:12][C:13]...,'CON(C(=O)C1=CC=C(C=C1)CNC(=O)OCC1=CC=CC=C1)C',Preparation of N-methoxy-N-methyl(4-{[(phenylm...,0.30,petroleum ether (2-methylpentane),EtOAc,50.0,50.0,NaN,NaN,100.0,CCCC(C)C,O=C(OCC)C
1,3,Putative nitrilase up-mutants were assayed in ...,[CH3:1][C:2]1(C)S[C@@H]2[C@H](NC([C@H](N)C3C=C...,'O[C@@H](CC(=O)O)CC#N',NaN,0.50,EtOAc,Hexane,50.0,50.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC
2,5,To 0.4 g N6-benzoyl-3′-O-(methylthiomethyl)-5′...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,'C(C1=CC=CC=C1)(=O)NC=1C=2N=CN([C@H]3C[C@H](OC...,NaN,0.30,EtOAc,Hexane,70.0,30.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC
3,6,3.5 g N4-benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[CH:30]=[CH:29][N:13]([C@@H...,'C(C1=CC=CC=C1)(=O)NC1=NC(N([C@H]2C[C@H](OCSC)...,NaN,0.40,EtOAc,Hexane,90.0,10.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC
4,7,To 0.5580 g N4-benzoyl-3′-O-(methylthiomethyl)...,[C:1]([NH:9][C:10]1[CH:33]=[CH:32][N:13]([C@@H...,'C(C1=CC=CC=C1)(=O)NC1=NC(N([C@H]2C[C@H](OCN=[...,NaN,0.50,EtOAc,Hexane,5.0,95.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC
5,17,"2,2-Dimethylpropylmagnesium chloride (1 M-diet...",[CH3:1][C:2]([CH3:7])([CH3:6])[CH2:3][Mg]Cl.[C...,'C(C)(C)(C)C=1C=C(C=CC1)NC(C1=C(N=CC=C1)CC(C)(...,NaN,0.45,Toluene,EtOAc,67.0,33.0,NaN,NaN,100.0,Cc1ccccc1,O=C(OCC)C
6,18,2-(Difluoro(4-fluorophenyl)methyl)-6-methylthi...,[F:1][C:2]([F:21])([C:14]1[CH:19]=[CH:18][C:17...,'ClC=1C2=C(N=C(N1)C(C1=CC=C(C=C1)F)(F)F)SC(=C2)C',NaN,0.70,EtOAc,Hexane,30.0,70.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC
7,28,NaH (58.1 mmol) was added to a solution of 1H-...,[H-].[Na+].[CH3:3][O:4][C:5]([C:7]1[CH:11]=[CH...,'COC(=O)C1=CN(C=C1)C(C)C',Step D5: 1-Isopropyl-1H-pyrrole-3-carboxylic a...,0.22,EtOAc,heptane,16.0,83.0,NaN,NaN,99.0,O=C(OCC)C,CCCCCCC
8,30,Ra—Ni (EtOH) (230 mmol) was added to a solutio...,[Cl:1][C:2]1[C:3](=[O:12])[N:4]([CH3:11])[CH:5...,'NC=1C=C(C(N(C1)C)=O)Cl',Step E5: 5-Amino-3-chloro-1-methyl-1H-pyridin-...,0.28,DCM,MeOH,90.0,10.0,NaN,NaN,100.0,ClCCl,CO
9,35,A saturated aqueous solution of NH4Cl (99 mL) ...,[NH4+].[Cl-].[Cl:3][C:4]1[C:5](=[O:22])[N:6]([...,'NC=1C=C(C(N(C1)CC1=CC=C(C=C1)OC)=O)Cl',Step O4: 5-Amino-3-chloro-1-(4-methoxy-benzyl)...,0.53,DCM,MeOH,96.0,3.0,NaN,NaN,99.0,ClCCl,CO


### 4.3 Processing of the SMILES
The last step of the processing attacks the smiles column. For the machine learning model, we need some useful features that can be fed and thus need to convert the SMILES into a uniform form. While working on the processing of the product SMILES, we found that there are some issues that need to be solved:
 - some product SMILES represents salts. As performing a TLC with salts would not make sense and the necessary information for the workup to convert the salts into the neutral molecule was not extracted, feeding our model with salts and correlating a Rf value could bias the results. That's why we decided to also drop the entries with salts.
 - some entries contain not only one SMILES but a list. This is due to the formation of isomers within the reaction. In the case of enantiomers, there is no divergence in the Rf values. However, the Rf values for diastereoisomers of small molecules can vary, depending on their interactions with the stationary phase. Thus, we need to check, if, in the case of multiple products, they are enantiomers and drop those entries in case they are not. 
 - for enantiomers, we need to remove the stereocentre on the atom which determines the enantiomers as we figured out, it would be easier for the model to learn those. Another approach could have been to double the entry and assign one enantiomer to each entry. This could be a potnetial outlook.
 - we extracted the product SMILES from the reaction SMILES to verify, if the product SMILES had been correctly extracted from the XML file. It turned out, that the reactions were atom-mapped, so also the mapping had to be removed.
 - lastly, of course all SMILES need to be canonicalized for internal comparison
 

In [5]:
def remove_salts(df: pd.DataFrame, column_name: str):
    """Checks if SMILES is a salt and removes it in this case.

    Args:
        df (pd.DataFrame): pre-cleaned dataframe
        column_name (str): productSmiles column

    Returns:
        df (pd.DataFrame): dataframe without rows that include salts
    """
    # Find the indices of rows where the SMILES list contains a salt
    indices_false_Rf = df[df[column_name].apply(lambda smiles_list: any('.' in smiles for smiles in smiles_list))].index 

    # Drop these rows from the DataFrame
    df.drop(indices_false_Rf, inplace=True)
    print(f"Number of rows dropped: {len(indices_false_Rf)}")
    print(f"Percentage of rows dropped: {len(indices_false_Rf) / df.shape[0] * 100}%")

    return df

In [6]:
def clean_smiles(df: pd.DataFrame, column_name: str):
    """Cleanes SMILES as apparently there are quotes around the SMILES which makes it impossible for rDKit to parse. Canonicalizes smiles

    Args:
        df (pd.DataFrame): dataframe cleaned with all previous functions
        column_name (str): productSmiles column

    Returns:
        df: pd.DataFrame: cleaned dataframe with SMILES that can be parsed in the form of a list in the column_name column
    """
    # Remove any leading or trailing whitespace
    df[column_name] = df[column_name].str.strip()
    
    # Remove any single or double quotes
    df[column_name] = df[column_name].str.replace("'", "").replace('"', '')
    
    # Split the string into a list of SMILES if there are multiple SMILES separated by a comma
    df[column_name] = df[column_name].str.split(',')
    
    # Canonicalize the SMILES
    df[column_name] = df[column_name].apply(lambda smiles_list: [Chem.MolToSmiles(Chem.MolFromSmiles(smiles)) for smiles in smiles_list])
    
    # Remove rows that contain salts
    df = remove_salts(df, column_name)
    
    return df

In [7]:
def remove_atom_mapping(mol: Chem.Mol):
    """Function that removes atom mapping from SMILES strings.

    Args:
        mol (rdkit.Chem.rdchem.Mol): Molecule with atom mapping
        
    Returns:
        smiles (str): SMILES string without atom mapping
    """
    # Remove atom mapping
    [atom.ClearProp('molAtomMapNumber') for atom in mol.GetAtoms() if atom.HasProp('molAtomMapNumber')]
    
    # Return canonical SMILES string
    return Chem.MolToSmiles(mol)
    

In [8]:
def extract_product_smiles(df: pd.DataFrame):
    """function that extracts the product side from the reaction SMILES in the dataframe and puts it into a new column, which then is cleaned by the clean_smiles function

    Args:
        df (pd.DataFrame): dataframe cleaned with all previous functions
    """
    df['extracted_pr_Smiles'] = None
    
    for index, row in df.iterrows():
        reaction_smiles = row['reactionSmiles']
        rxn = rdChemReactions.ReactionFromSmarts(reaction_smiles)
        
        #Extract, clean and canonicalize products
        products = [remove_atom_mapping(mol) for mol in rxn.GetProducts()]
        
        #canonicalize the SMILES
        products = products = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles)) for smiles in products]
        
        df.at[index, 'extracted_pr_Smiles'] = products
        
    return df

In [9]:
def are_enantiomers(mol1: Chem.Mol, mol2: Chem.Mol):
    """
    Function that checks if two molecules are stereoisomers. Credits to: https://github.com/rdkit/rdkit/discussions/7169 for the inspiration with the swapping of the "@" and "@@" in the SMILES strings.

    Args:
        mol1 (Chem.Mol): RDKit molecule object 1
        mol2 (Chem.Mol): RDKit molecule object 2

    Returns:
        bool: True if the molecules are enantiomers, False otherwise
        int: Index of the chiral atom if the molecules are enantiomers, None otherwise
    """
    # If either SMILES string is invalid, return False
    if mol1 is None or mol2 is None:
        return False, None

    # Convert the molecule objects back to canonical SMILES strings
    can_smi1 = Chem.MolToSmiles(mol1)
    can_smi2 = Chem.MolToSmiles(mol2)

    # If either SMILES string does not contain "@", return False
    if "@" not in can_smi1 or "@" not in can_smi2:
        return False, None

    # Swap "@" and "@@" in can_smi1
    swapped_can_smi1 = can_smi1.replace("@@", "__DOUBLE_AT__").replace("@", "@@").replace("__DOUBLE_AT__", "@")

    # If the swapped can_smi1 is equal to can_smi2, return True
    if swapped_can_smi1 == can_smi2:
        # Find the index of the chiral atom
        for atom in mol1.GetAtoms():
            if atom.GetChiralTag() != Chem.rdchem.ChiralType.CHI_UNSPECIFIED:
                return True, atom.GetIdx()
    
    # If none of the above conditions were met, return False
    return False, None

In [10]:
def process_smiles(df: pd.DataFrame):
    """Function that processes the extracted product smiles from the rxn smiles. 

    Args:
        df (pd.DataFrame): _description_
    """
    rows_to_drop = []
    for index, row in df.iterrows():
        rxn_smiles_extr = row['extracted_pr_Smiles']

        if len(rxn_smiles_extr) == 1:
            mol = Chem.MolFromSmiles(rxn_smiles_extr[0])
            if mol is not None:
                df.at[index, 'productSmiles_processed'] = Chem.MolToSmiles(mol, isomericSmiles=True)
            else:
                rows_to_drop.append(index)
        elif len(rxn_smiles_extr) == 2:
            mol1 = Chem.MolFromSmiles(rxn_smiles_extr[0])
            mol2 = Chem.MolFromSmiles(rxn_smiles_extr[1])
            are_enantiomers_result, chiral_atom_index = are_enantiomers(mol1, mol2)
            if mol1 is not None and mol2 is not None and are_enantiomers_result:
                mol1.GetAtomWithIdx(chiral_atom_index).SetChiralTag(Chem.rdchem.ChiralType.CHI_UNSPECIFIED)
                df.at[index, 'productSmiles_processed'] = Chem.MolToSmiles(mol1, isomericSmiles=True)
            else:
                rows_to_drop.append(index)
                
    df.drop(rows_to_drop, inplace=True)
    print(f"Number of rows dropped: {len(rows_to_drop)}")
    print(f"Percentage of rows dropped: {len(rows_to_drop) / df.shape[0] * 100}%")
    
    return df

And now apply those function to our large dataset:

In [30]:
# clean SMILES and extract product SMILES
df = clean_smiles(df, 'productSmiles')
df = extract_product_smiles(df)
df = process_smiles(df)
df.head()

Number of rows dropped: 267
Percentage of rows dropped: 1.7640063424947146%
Number of rows dropped: 73
Percentage of rows dropped: 0.4846312155613092%


,Unnamed: 0,paragraphText,reactionSmiles,productSmiles,title,Rf,Solvent_A,Solvent_B,Percent_A,Percent_B,Additive_C,Percent_C,sum,Solvent_A_Smiles,Solvent_B_Smiles,extracted_pr_Smiles,productSmiles_processed
0,2,A solution of 4-{[(phenylmethoxy)carbonylamino...,[C:1]1([CH2:7][O:8][C:9]([NH:11][CH2:12][C:13]...,[CON(C)C(=O)c1ccc(CNC(=O)OCc2ccccc2)cc1],Preparation of N-methoxy-N-methyl(4-{[(phenylm...,0.3,petroleum ether (2-methylpentane),EtOAc,50.0,50.0,NaN,NaN,100.0,CCCC(C)C,O=C(OCC)C,[CON(C)C(=O)c1ccc(CNC(=O)OCc2ccccc2)cc1],CON(C)C(=O)c1ccc(CNC(=O)OCc2ccccc2)cc1
1,3,Putative nitrilase up-mutants were assayed in ...,[CH3:1][C:2]1(C)S[C@@H]2[C@H](NC([C@H](N)C3C=C...,[N#CC[C@@H](O)CC(=O)O],NaN,0.5,EtOAc,Hexane,50.0,50.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC,[N#CC[C@@H](O)CC(=O)O],N#CC[C@@H](O)CC(=O)O
2,5,To 0.4 g N6-benzoyl-3′-O-(methylthiomethyl)-5′...,[C:1]([NH:9][C:10]1[C:11]2[N:12]=[CH:13][N:14]...,[[N-]=[N+]=NCO[C@H]1C[C@H](n2cnc3c(NC(=O)c4ccc...,NaN,0.3,EtOAc,Hexane,70.0,30.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC,[[N-]=[N+]=NCO[C@H]1C[C@H](n2cnc3c(NC(=O)c4ccc...,[N-]=[N+]=NCO[C@H]1C[C@H](n2cnc3c(NC(=O)c4cccc...
3,6,3.5 g N4-benzoyl-5′-O-tert-butyldimethylsilyl-...,[C:1]([NH:9][C:10]1[CH:30]=[CH:29][N:13]([C@@H...,[CSCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc3)nc2=O)O...,NaN,0.4,EtOAc,Hexane,90.0,10.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC,[CSCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc3)nc2=O)O...,CSCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc3)nc2=O)O[...
4,7,To 0.5580 g N4-benzoyl-3′-O-(methylthiomethyl)...,[C:1]([NH:9][C:10]1[CH:33]=[CH:32][N:13]([C@@H...,[[N-]=[N+]=NCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc...,NaN,0.5,EtOAc,Hexane,5.0,95.0,NaN,NaN,100.0,O=C(OCC)C,CCCCCC,[[N-]=[N+]=NCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc...,[N-]=[N+]=NCO[C@H]1C[C@H](n2ccc(NC(=O)c3ccccc3...


In [40]:
df.shape

(15063, 17)

In [31]:
df.to_csv(r'C:\Users\milen\git\ppChem\PPChem_TLC\data\dataset_cleaned.csv', index=False)

From our first 2 million extracted reactions we cut down everything to 15 thousand datapoints. That was a long way (pretty condensed in this notebook) and we faced many challenged along the way, first through messing up with regex, going over to our LLM, cleaning up the SMILEs and with every step getting more familiar with the data, overall noticing, that the process of data-processing can take more time than implementing the final machine learning model. 

# 5. Preparing of the Data for Training 🏋️
Finally we got more or less everything we need to go on. For the machine learning model we need some features that can be fed into the model. To train the model, we needed to convert it into some machine-readable format. This and the trainings of different models are described in the second notebook. 